# MODULE 2A: EXTRACT FEATURES FROM RENDERED IMAGES (2D FEATURE VECTOR)

Using DINOv2 we extract features of rendered images

In [ ]:
import glob
import os
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
import torch

# Load processor and model
print("Loading model and processor...")
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base").eval()
print("Loaded model and processor.")

# Load all render image files
render_files = sorted(glob.glob("output/renders/*.png"))
print(f"Found {len(render_files)} images")

all_features = []

for i, file in enumerate(render_files):
    if not os.path.isfile(file):
        print(f"⚠️ Skipping non-file: {file}")
        continue

    try:
        img = Image.open(file).convert("RGB")
    except Exception as e:
        print(f"Could not open image {file}: {e}")
        continue

    inputs = processor(images=img, return_tensors="pt")

    with torch.no_grad():
        output = model(**inputs)
        feats = output.last_hidden_state  # [1, tokens, dim]

    if feats is not None and feats.numel() > 0:
        all_features.append(feats[0])  # shape: [tokens, dim]
    else:
        print(f"Empty features for image {i}: {file}")

# Final stacking
if not all_features:
    raise ValueError("No features extracted. Please check your images or rendering.")
else:
    all_features_tensor = torch.stack(all_features, dim=0)
    print(f"Final feature tensor shape: {all_features_tensor.shape}")
    for i in range(len(all_features_tensor)):
      print(f"Feature tensor {i}: {all_features_tensor[i][0][:5]}")
